In [1]:
from load_matlab_elmannet import *
import numpy as np
import gym
import time
import os
from PIL import Image

In [2]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [3]:
sleep_time = 0.05
model_path = "../2-elman_train/models/play_45to49f_1to1000epoch10000h_500/playm_45to49f_1to1000epoch_10000h_500.mat"

match = 45
data_path = f"../1-generate/data/match_" + str(match) + "/"

In [4]:
ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
}

In [5]:
"""ACTIONS = {
    "right": 2,
    "left": 3,
}"""

'ACTIONS = {\n    "right": 2,\n    "left": 3,\n}'

In [5]:
ACTIONS_LIST = list(ACTIONS.values())

In [30]:
def load_npz(m):
    path = data_path + "npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [31]:
num_of_frames, frames, actions, rewards, lifes = load_npz(match)

Successfully loaded NPZ.


In [32]:
frames = frames.reshape(len(frames), -1)
frames = frames[:1000]

In [33]:
frames.shape

(1000, 20400)

In [34]:
def prepare_action_data(action):

    new_action = np.zeros((1, len(ACTIONS)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [35]:
actions

array([0, 0, 0, ..., 3, 0, 0])

In [36]:
targets = [prepare_action_data(actions[i])[0] for i in range(len(actions))]

In [37]:
targets = np.array(targets)

In [38]:
model = LoadElman(model_path, True)

U shape: (500, 20400)
W shape: (500, 500)
V shape: (9, 500)
bh shape: (500,)
bo shape: (9,)


In [39]:
model.reset_prev_s()

print_class = False

for i in range(len(frames)):
    if(print_class):
        print(i+1, np.argmax(targets[i]), "->", np.argmax(model.predict_step(frames[i]/255)))
    else:
        print(i+1, targets[i], "->", model.predict_step(frames[i]/255))

1 [1 0 0 0 0 0 0 0 0] -> [9.54964388e-01 3.80129393e-02 3.60240038e-04 1.78451026e-03
 6.74139213e-03 6.40003927e-06 3.07310179e-03 3.96573580e-03
 1.67736829e-03]
2 [1 0 0 0 0 0 0 0 0] -> [8.78169626e-01 1.37863459e-01 2.93946495e-04 2.52854296e-03
 2.57199200e-03 1.62049694e-06 5.34943261e-03 3.86678418e-03
 5.50152418e-04]
3 [1 0 0 0 0 0 0 0 0] -> [8.36408073e-01 1.53102329e-01 2.14801537e-04 1.11107171e-03
 2.05415657e-03 1.58192776e-06 5.90292317e-03 2.61804598e-03
 8.26658195e-04]
4 [1 0 0 0 0 0 0 0 0] -> [8.25858050e-01 1.79664892e-01 2.05325637e-04 1.24663560e-03
 1.95215004e-03 1.57082053e-06 6.23821689e-03 2.34103676e-03
 9.34877797e-04]
5 [1 0 0 0 0 0 0 0 0] -> [8.21700375e-01 2.51022689e-01 2.43289204e-04 2.94874450e-03
 2.19429984e-03 1.38803275e-06 4.92817755e-03 3.07125137e-03
 5.21870233e-04]
6 [1 0 0 0 0 0 0 0 0] -> [8.39723604e-01 2.37640740e-01 2.50078433e-04 3.07238396e-03
 2.56882888e-03 1.39445816e-06 5.24761063e-03 3.46231311e-03
 4.85998109e-04]
7 [1 0 0 0 0 0 0

51 [0 1 0 0 0 0 0 0 0] -> [4.30698165e-02 9.56233109e-01 2.72302127e-04 3.19404022e-03
 1.41546296e-03 2.16337758e-06 2.36893259e-03 2.44420222e-03
 3.16403176e-04]
52 [0 1 0 0 0 0 0 0 0] -> [5.90737987e-02 9.52054819e-01 3.26496590e-04 4.62724374e-03
 1.27016074e-03 1.81064797e-06 1.96847605e-03 3.42524866e-03
 2.44403127e-04]
53 [0 1 0 0 0 0 0 0 0] -> [8.12050375e-02 9.17227398e-01 3.41202738e-04 2.00019529e-03
 6.90408585e-04 1.19861817e-06 2.00721668e-03 2.47039621e-03
 9.45569745e-04]
54 [0 1 0 0 0 0 0 0 0] -> [8.36441028e-02 9.06995161e-01 2.64613123e-04 1.43016100e-03
 8.66591105e-04 1.37210482e-06 2.61935282e-03 1.52512631e-03
 1.07318391e-03]
55 [0 1 0 0 0 0 0 0 0] -> [6.49461547e-02 9.08161599e-01 2.49573040e-04 3.02887194e-03
 1.49812760e-03 1.39736122e-06 2.32065325e-03 2.07671970e-03
 4.08263387e-04]
56 [0 1 0 0 0 0 0 0 0] -> [4.76404667e-02 9.45729617e-01 2.13695426e-04 2.76298147e-03
 7.74568851e-04 9.34354447e-07 2.48753011e-03 1.41715331e-03
 1.12869409e-03]
57 [0 1 0 

111 [0 1 0 0 0 0 0 0 0] -> [1.46880743e-01 8.71915942e-01 3.99833380e-04 9.39239677e-04
 7.94619557e-04 8.21651139e-07 2.78254411e-03 1.74928164e-03
 1.08267942e-03]
112 [0 1 0 0 0 0 0 0 0] -> [9.65545289e-02 9.03367253e-01 3.77357017e-04 1.20764544e-03
 5.20328502e-04 7.09761446e-07 1.46197203e-03 1.96783111e-03
 6.45517639e-04]
113 [0 1 0 0 0 0 0 0 0] -> [1.11265828e-01 8.83663025e-01 5.61125197e-04 1.42466536e-03
 4.73200484e-04 8.53128319e-07 2.22167277e-03 3.12288966e-03
 5.14681762e-04]
114 [0 1 0 0 0 0 0 0 0] -> [1.26457696e-01 8.07173834e-01 2.09563887e-04 1.27763482e-03
 4.36332721e-04 7.03971429e-07 3.27458574e-03 1.07457990e-03
 5.61411482e-04]
115 [0 1 0 0 0 0 0 0 0] -> [1.61359846e-01 8.27686836e-01 2.18077461e-04 3.17549984e-03
 7.58678335e-04 6.25734900e-07 2.52634725e-03 2.38517957e-03
 6.99670826e-04]
116 [0 1 0 0 0 0 0 0 0] -> [1.35158647e-01 8.11949599e-01 3.74028645e-04 1.58815700e-03
 9.55691724e-04 7.81319416e-07 1.65990166e-03 1.89642796e-03
 6.66049458e-04]
117 

172 [1 0 0 0 0 0 0 0 0] -> [4.75362408e-01 4.64267069e-01 1.55694852e-04 1.88261186e-03
 1.45651128e-03 7.76964094e-07 1.34355815e-03 2.18447721e-03
 4.62820318e-04]
173 [1 0 0 0 0 0 0 0 0] -> [6.20443400e-01 3.61686192e-01 1.81058620e-04 9.83764773e-04
 9.55185797e-04 8.96709260e-07 2.63500970e-03 2.32577521e-03
 6.04359796e-04]
174 [1 0 0 0 0 0 0 0 0] -> [5.54530316e-01 3.16018349e-01 1.21182030e-04 9.21662776e-04
 5.42437836e-04 8.91907987e-07 2.73226911e-03 2.02933744e-03
 6.79750099e-04]
175 [0 0 1 0 0 0 0 0 0] -> [4.16668080e-01 3.57002587e-01 1.70742242e-04 1.62426455e-03
 1.60126368e-03 6.62197838e-07 2.23073059e-03 2.21951091e-03
 7.09599052e-04]
176 [1 0 0 0 0 0 0 0 0] -> [7.23701770e-01 1.86292800e-01 1.01853216e-04 8.42548693e-04
 1.32582152e-03 9.04190427e-07 7.34701209e-03 2.52591880e-03
 1.18525026e-03]
177 [1 0 0 0 0 0 0 0 0] -> [7.55858638e-01 1.07461623e-01 5.76064537e-05 2.75650548e-03
 2.55789017e-03 8.87404947e-07 4.32846849e-03 1.99474379e-03
 9.27910094e-04]
178 

224 [0 0 1 0 0 0 0 0 0] -> [5.90013605e-01 4.61666435e-02 2.54220215e-04 2.35053913e-03
 2.00943641e-04 1.02492975e-06 3.84774754e-03 3.57534017e-03
 3.05990965e-04]
225 [1 0 0 0 0 0 0 0 0] -> [5.72074745e-01 7.42698337e-02 1.21076651e-04 2.95682784e-03
 2.49568998e-04 7.63282529e-07 4.35001056e-03 3.73079904e-03
 9.08341862e-04]
226 [1 0 0 0 0 0 0 0 0] -> [6.71163269e-01 3.94062233e-02 1.55097350e-04 1.42074986e-03
 4.16855968e-04 1.16801031e-06 9.66922138e-03 4.63805171e-03
 7.79688115e-04]
227 [1 0 0 0 0 0 0 0 0] -> [7.37679321e-01 4.21009682e-02 2.06966835e-04 2.78844213e-03
 1.63700555e-04 1.19573962e-06 8.20227317e-03 6.66417653e-03
 7.28034083e-04]
228 [1 0 0 0 0 0 0 0 0] -> [5.94278920e-01 3.58483067e-02 1.77074006e-04 1.60815831e-03
 3.04835010e-04 1.45002996e-06 1.32231373e-02 4.90451360e-03
 5.35178812e-04]
229 [0 0 1 0 0 0 0 0 0] -> [5.40581788e-01 3.10304322e-02 2.77378781e-04 1.53240204e-03
 3.73128072e-04 2.30523143e-06 1.10761242e-02 2.25275406e-03
 3.70700508e-04]
230 

...

721 [1 0 0 0 0 0 0 0 0] -> [5.73371045e-01 1.01699547e-01 1.57913390e-04 1.81137482e-04
 3.97380650e-05 7.91694408e-07 7.55421587e-03 4.94698202e-03
 6.79410176e-05]
722 [1 0 0 0 0 0 0 0 0] -> [5.58791277e-01 1.03455757e-01 3.90159260e-04 9.88151649e-05
 3.96039107e-05 8.44129101e-07 9.88409004e-03 7.97273023e-03
 8.74388442e-05]
723 [0 0 1 0 0 0 0 0 0] -> [5.32383952e-01 1.29843859e-01 3.70204014e-04 1.00894080e-04
 3.61867620e-05 1.37042708e-06 2.84041665e-02 7.13278714e-03
 6.96433393e-05]
724 [1 0 0 0 0 0 0 0 0] -> [6.89241594e-01 1.80661170e-01 4.72183838e-05 1.11697094e-04
 3.47730447e-05 2.53888088e-06 7.73429219e-03 6.57415667e-03
 1.30745035e-04]
725 [1 0 0 0 0 0 0 0 0] -> [6.81109367e-01 1.22566369e-01 1.89131117e-04 1.77797166e-04
 3.66990955e-05 3.20730725e-06 7.60625401e-03 4.13836147e-03
 6.99649238e-05]
726 [0 0 0 1 0 0 0 0 0] -> [6.62542594e-01 9.69924948e-02 2.60888274e-04 1.64719473e-04
 3.26382006e-05 3.15254739e-06 9.93805333e-03 3.09066830e-03
 8.91703860e-05]
727 

771 [1 0 0 0 0 0 0 0 0] -> [8.57432104e-01 1.45906298e-02 6.67832575e-05 1.18066391e-04
 4.87093753e-05 4.05789497e-06 1.68719214e-02 4.54385259e-03
 9.40394091e-05]
772 [1 0 0 0 0 0 0 0 0] -> [8.70900517e-01 1.81078335e-02 8.48028425e-05 2.06940827e-04
 4.44703528e-05 3.92111806e-06 8.06567161e-03 2.91267334e-03
 9.90041006e-05]
773 [1 0 0 0 0 0 0 0 0] -> [8.73374115e-01 1.85583480e-02 7.77881489e-05 1.63355474e-04
 5.65371194e-05 4.08420342e-06 9.06640108e-03 2.52966742e-03
 1.05430348e-04]
774 [1 0 0 0 0 0 0 0 0] -> [8.25668854e-01 2.63637290e-02 9.75152123e-05 8.96520476e-05
 5.62523709e-05 4.20870941e-06 9.26755510e-03 2.12413109e-03
 7.66753664e-05]
775 [1 0 0 0 0 0 0 0 0] -> [9.12932392e-01 1.46669065e-02 8.56784554e-05 1.57777718e-04
 4.93252472e-05 3.15415560e-06 9.69262492e-03 2.81339318e-03
 1.06823036e-04]
776 [1 0 0 0 0 0 0 0 0] -> [8.66962974e-01 1.69800672e-02 1.47274682e-04 1.42755453e-04
 4.31099774e-05 2.29743759e-06 8.01733882e-03 1.58656954e-03
 9.38777398e-05]
777 

836 [1 0 0 0 0 0 0 0 0] -> [5.95233322e-01 5.41122300e-02 1.86631448e-04 1.03328532e-04
 2.44715619e-05 2.15835083e-06 3.34358140e-03 4.59772973e-03
 1.44759522e-04]
837 [0 0 0 1 0 0 0 0 0] -> [5.00647117e-01 6.42084308e-02 1.65797335e-04 7.43178852e-05
 3.30386452e-05 2.74942434e-06 7.43490684e-03 2.76594006e-03
 8.06949342e-05]
838 [0 0 0 1 0 0 0 0 0] -> [6.34589406e-01 4.66402462e-02 2.13436094e-04 1.70268231e-04
 1.70409169e-05 2.94330542e-06 5.23591832e-03 2.74648310e-03
 3.13241661e-04]
839 [0 0 0 1 0 0 0 0 0] -> [6.46277795e-01 7.09242610e-02 1.88730815e-04 2.22682361e-04
 3.75330904e-05 2.06152584e-06 5.80811797e-03 9.29360831e-03
 1.14642306e-04]
840 [0 0 0 1 0 0 0 0 0] -> [2.83952018e-01 1.71204328e-01 9.29801836e-05 2.08481322e-04
 5.02593430e-05 1.16115776e-06 1.61848827e-03 6.83688527e-03
 1.09115707e-04]
841 [0 0 0 1 0 0 0 0 0] -> [6.80658394e-01 4.66708724e-02 4.53357864e-04 2.87362000e-04
 3.22036444e-05 1.29425806e-06 1.29317629e-03 2.64945053e-03
 1.86043282e-04]
842 

886 [1 0 0 0 0 0 0 0 0] -> [3.17476388e-01 1.39577637e-01 6.35030760e-04 6.65124661e-05
 1.92751359e-05 2.32757046e-06 8.97919220e-03 5.01001786e-03
 1.42292393e-04]
887 [1 0 0 0 0 0 0 0 0] -> [5.49854291e-01 1.52536590e-01 8.36057289e-04 1.04207540e-04
 2.71252573e-05 2.33341805e-06 6.25875073e-03 1.79722291e-02
 3.57929115e-05]
888 [1 0 0 0 0 0 0 0 0] -> [5.02050423e-01 1.11431745e-01 5.62370660e-04 3.07652242e-04
 2.14502573e-05 1.48964381e-06 6.67955595e-03 9.80019466e-03
 5.88663087e-05]
889 [1 0 0 0 0 0 0 0 0] -> [2.77017011e-01 1.78266519e-01 4.42975617e-04 3.66322938e-05
 2.76677639e-05 1.07441148e-06 6.63985852e-03 1.28325855e-02
 1.19236322e-04]
890 [1 0 0 0 0 0 0 0 0] -> [5.19451308e-01 7.02494428e-02 1.01333109e-04 2.03223782e-04
 1.72568276e-05 2.86180787e-06 6.53628739e-03 1.31374962e-02
 1.09494183e-04]
891 [1 0 0 0 0 0 0 0 0] -> [4.51726528e-01 1.29816658e-01 1.59308835e-04 1.46485419e-04
 1.08110596e-05 1.47209491e-06 8.95661475e-03 2.68669897e-03
 1.82315721e-04]
892 

946 [0 1 0 0 0 0 0 0 0] -> [3.45468058e-01 3.01341971e-01 4.97155164e-04 1.70080925e-04
 2.50513186e-05 1.77763086e-06 2.39516265e-03 1.50356043e-02
 4.72550773e-05]
947 [0 1 0 0 0 0 0 0 0] -> [3.69838850e-01 3.61509008e-01 6.49377069e-04 1.31699777e-04
 3.71932731e-05 1.06761610e-06 3.45132486e-03 9.78755820e-03
 9.04105614e-05]
948 [0 1 0 0 0 0 0 0 0] -> [3.53381871e-01 4.03960365e-01 7.41212968e-04 6.59581315e-05
 2.40152656e-05 9.88861652e-07 6.13437891e-03 9.48382279e-03
 8.52721625e-05]
949 [0 1 0 0 0 0 0 0 0] -> [4.38586727e-01 3.62738352e-01 5.93049522e-04 1.07686094e-04
 2.13858357e-05 9.44954826e-07 8.36370571e-03 6.17734491e-03
 1.66396373e-04]
950 [1 0 0 0 0 0 0 0 0] -> [3.63692144e-01 2.95075784e-01 9.06379010e-04 7.88842124e-05
 3.26787396e-05 6.40706540e-07 6.89631155e-03 6.39795090e-03
 1.59929101e-04]
951 [1 0 0 0 0 0 0 0 0] -> [3.78525606e-01 3.92798466e-01 1.30135506e-03 5.33866695e-05
 3.12030559e-05 8.82932862e-07 1.43145013e-02 5.30528870e-03
 1.37230037e-04]
952 

In [40]:
model.reset_prev_s()
predicted = []
correct = 0
wrong = 0
for i in range( len(frames) ):
    
    if np.argmax(targets[i]) == np.argmax(model.predict_step(frames[i]/255)):
        correct += 1
    else:
        wrong +=1

In [41]:
print(correct)
print(wrong)

616
384


In [ ]:
#798/(798+202)

In [43]:
env = gym.make("Enduro-v0")

In [44]:
frame = env.reset()

In [45]:
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

In [46]:
frame.shape

(210, 160, 3)

In [47]:
model.reset_prev_s()

In [48]:
while(True):
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(20400,)
    frame = frame/255
    
    action = model.predict_step(frame)
    action = list(ACTIONS.values())[np.argmax(action, axis=0)]
    
    print(action)
    
    frame, reward, done, info = env.step(action)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-48-7fe61428bda0>", line 14, in <module>
    action = model.predict_step(frame)
  File "/mnt/c/Users/alece/Desktop/UnB/tcc/rnn_enduro/3-load_model/load_matlab_elmannet.py", line 36, in predict_step
    mulu = np.dot(self.U, new_input)
  File "<__array_function__ internals>", line 5, in dot
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3

TypeError: object of type 'NoneType' has no len()